In [1]:
#Input Q

#############
#...........#
###D#C#A#B###
  #C#D#A#B#
  #########

p = [
    {'name': 'b', 'sub' : 2},
    {'name': 'a', 'sub' : 2},
    {'name': 'd', 'sub' : 2},
    {'name': 'c', 'sub' : 2},
    {'name': 'b', 'sub' : 1},
    {'name': 'a', 'sub' : 1},
    {'name': 'c', 'sub' : 1},
    {'name': 'd', 'sub' : 1}
]

In [487]:
# Test Q

#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########

p = [
    {'name': 'a', 'sub' : 1},
    {'name': 'c', 'sub' : 1},
    {'name': 'd', 'sub' : 1},
    {'name': 'a', 'sub' : 2},
    {'name': 'd', 'sub' : 2},
    {'name': 'b', 'sub' : 1},
    {'name': 'c', 'sub' : 2},
    {'name': 'b', 'sub' : 2}
]

In [488]:
import numpy as np
import pandas as pd
spaces = []
#types = 'corro': corridor that is not outside a room
#        'out'  : corridor that is outside a room
#        'room' : a room
c = {
    'type': 'corro',
    'name': 0,
    'sub': 0,
    'for': 0
}

o = {
    'type': 'out',
    'name': 0,
    'sub': 0,
    'for': 0
}


blank = [0,0]

for _ in range(2):
    spaces.append(c)
    
for _ in range(4):
    spaces.append(o)
    spaces.append(c)

spaces.append(c)

for _ in range(2):
    for f in ['a','b','c','d']:
        r = {
            'type': 'room',
            'name' : 0, 
            'sub' : 0,
            'for': f'{f}'
        }
        spaces.append(r)

for idx, pieces in enumerate(p):
    spaces[-1*(idx+1)]['name'] = pieces['name']
    spaces[-1*(idx+1)]['sub'] = pieces['sub']

df = pd.DataFrame(spaces.copy())
df.fillna(value=0, inplace=True)
pieces = pd.DataFrame(p.copy())
boards = []
df

,type,name,sub,for
0,corro,0,0,0
1,corro,0,0,0
2,out,0,0,0
3,corro,0,0,0
4,out,0,0,0
5,corro,0,0,0
6,out,0,0,0
7,corro,0,0,0
8,out,0,0,0
9,corro,0,0,0


In [507]:
def isFinished(board):
    return (df['name']==df['for']).all()

def roomToOut(room):
    moves = 0
    if room <= 10:
        return room, moves
    
    moves = (room // 15) + 1 
    
    out = room - 11
    out = out % 4
    out = out + 1
    out = out * 2    
    return out, moves

def calcScore(piece,start,end):
    if piece == 'a':
        price = 1
    elif piece == 'b':
        price = 10
    elif piece == 'c':
        price = 100
    elif piece == 'd':
        price = 1000
    else:
        raise ValueError(f"Wrong piece - {piece}")

    s = min(start,end)
    e = max(start,end)
    
    moves = 0
    c1,m1 = roomToOut(s)
    c2,m2 = roomToOut(e)
    # The 2 places are in the same room already
    if c1 == c2:
        moves = abs(m1-m2)
    else:
        moves = m1 + m2 + abs(c1-c2)
  
    return price * moves

def isInvalidRoom(b,p,empty):
    otherRoom = (empty.name>=15) * (empty.name-4) + (empty.name<15) * (empty.name + 4)
    try:
        isItForMe = b.loc[empty.name,'for']==p['name']
    except:
        print(b)
    isItEmpty = b.loc[otherRoom,'name']==0
    isMyFriendThere = b.loc[otherRoom,'name']==p['name']
    result = isItForMe & (isItEmpty | isMyFriendThere)
    return not(result)


def availableMoves(boards, bestScore=1e99):
    newBoards = []
    lineages = np.array(boards,dtype=object).T.tolist()[2]
    if len(lineages)==1:
        eldest=0
    else:
        eldest = np.max([int(j) for i in lineages for j in i.split('-')])
    
    moveCounter = eldest
    
    for current in boards:
        board, score, parent = current
        
        child = parent + '-'
   
        for empty in board[(board['name']==0) & (board['type'] != 'out')].iloc():
            for p in pieces.iloc():
                if empty['type'] == 'room' and isInvalidRoom(board,p,empty):
                    continue

                newBoard = board.copy()

                start = newBoard.loc[((newBoard['name']==p['name']) & (newBoard['sub']==p['sub']))].index[0]
                end = empty.name

                #cannot move from a corriodor to another corridor
                if start <= 10 and end <= 10:
                    continue

                score = score + calcScore(p['name'],start,end)

                newBoard.loc[((newBoard['name']==p['name']) & (newBoard['sub']==p['sub'])),p.keys()] = blank
                newBoard.loc[empty.name,p.keys()] = p

                if isFinished(newBoard) and score < bestScore:
                    bestScore = score

                allGood = True
                for idx,h in enumerate(boards):
                    # only check the names, don't care if a1 & a2 are swapped around
                    if (len(h[0]['name'].compare(newBoard['name']))):                        
                        continue
                    elif score < h[1]:
                        mr = []
                        for i in h[2].split('-'):
                            mr.append(int(i))
                        mostRecent = np.max(mr)
                        #mostRecent = np.max([int(j) for i in h[2] for j in i.split('-')])
                        diff = h[1] - score
                        counter = 0
                        for i, oh in enumerate(boards):
                            if (mostRecent in [int(z) for z in oh[2].split('-')]):
                                counter = counter + 1
                                boards[i][1] = boards[i][1] - diff
                        assert counter > 0, f"{h[2]}"
                    
                    allGood = False
                    break

                if allGood:
                    moveCounter = moveCounter + 1
                    newBoards.append([newBoard,score, f"{child}{moveCounter}"])

    return newBoards, bestScore

In [503]:
h = ['0-1-99']
np.max([int(j) for i in h for j in i.split('-')])

99

In [508]:
#current = newCurrent.copy()#[[df,0,'0'],]
for idx in range(8):
    newCurrent, bestScore = availableMoves(current,bestScore)
    print(len(current), len(newCurrent))
    current = newCurrent

0-40-1765
0-48-2107
0-56-2449
0-14-653
0-22-995
0-30-1337
0-38-1679
0-46-2021
0-54-2363
0-9-441
0-17-783
0-25-1125
0-33-1467
0-41-1809
0-49-2151


IOStream.flush timed out


0-16-740
0-24-1082
0-32-1424
0-40-1766
0-48-2108
0-56-2450
0-9-442
0-17-784
0-25-1126
0-33-1468
0-41-1810
0-49-2152
0-6-311
0-6-312


IOStream.flush timed out
IOStream.flush timed out


0-14-654


IOStream.flush timed out


0-22-996
0-30-1338
0-38-1680
0-46-2022
0-54-2364
0-8-397
0-8-398


IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


KeyboardInterrupt: 

In [ ]:
1

In [ ]:

df = pd.DataFrame(spaces.copy())
boards = []
boards.append([df,0])
min_cost = 9999999999999999999999999999

for wtf in range(10):
    print(wtf, len(boards))
    temp_board = []
    for board,cost in boards:
        moves = []
        for piece_type in ['a','b','c','d']:
            for piece_num in range(1,3):
                piece = f'{piece_type}{piece_num}'
                moves += [[piece, i] for i in available_moves(board,piece)]
        for p,m in moves:
            tb = board.copy()
            b, c = make_move(tb,p,m)
            cost += c
            if is_winning(b):
                if cost < min_cost:
                    min_cost = cost
                    continue
            
            add_me = True
            for i,(t,c) in enumerate(temp_board):
                if (t==b).all().all():
                    assert 3==2,'work out what to keep'
                    add_me = False
            if add_me:
                temp_board.append([b.copy(),cost])
    boards = temp_board.copy()
min_cost

0 1
1 56
2 2541


In [ ]:
161574 - too high

In [ ]:


def is_home(board, piece):
    return len(board[((board['type'] == 'room') & 
                      (board['status'] == piece) & 
                      (board['for'] == piece[0]))])

def available_moves(board,piece):
    if is_home(board,piece):
        return []    
    moves = []

    moves += [i for i in board[((board['type'] == 'corro') & (!board['status']))].index.values] #if corrodior
    
    d1 = board[((board['type'] == 'room') & (!board['status']['name']) & (board['for'] == piece[0]))]
    if len(d1) == 2: #if both room spots are empty
        moves += [i for i in d1.index.values]
    if len(d1) == 1: #if one is empty & the other has the right one in it
        d2 = board[((board['type'] == 'room') &  (board['for'] == piece[0]))]
        if np.array([i[0] == piece[0] for i in d2['status'].values]).any():
            moves += [i for i in d2[(d2['status'] == '.')].index.values]
    return moves

def is_winning(board):
    counter = 0
    for piece_type in ['a','b','c','d']:
        for piece_num in range(1,3):
            piece = f'{piece_type}{piece_num}'
            counter += is_home(board,piece)
    return counter == 8


def make_move(board,piece,move):
    assert board.loc[move]['status'] == '.', "can't sit here; seats taken!"
    
    from_idx = board[(board['status']==piece)].index[0]
    board.iloc[from_idx]['status'] = '.'
    board.iloc[move]['status'] = piece
    if 'a' in piece:
        cost = 1
    elif 'b' in piece:
        cost = 10
    elif 'c' in piece:
        cost = 100
    elif 'd' in piece:
        cost = 1000
    else:
        assert 3==2, 'broke it'
    return board, cost